In [28]:
# =============================================================================
# 1. CARREGAMENTO E PRÉ-PROCESSAMENTO DOS DADOS
# =============================================================================

import pandas as pd
import gspread
import google.auth
from pathlib import Path

# Carrega dados base
df_fatores = pd.read_csv("Fatores.csv")
df_relacao = pd.read_csv("relacao_fatores.csv")

# Normaliza textos
def norm(s): return ' '.join(str(s).strip().split()).replace('\n', ' ')
df_fatores['Fator'] = df_fatores['Fator'].apply(norm)
df_relacao.rename(columns={df_relacao.columns[0]: 'Fatores'}, inplace=True)
df_relacao['Fatores'] = df_relacao['Fatores'].apply(norm)
df_relacao.columns = [c if c=='Fatores' else norm(c) for c in df_relacao.columns]

# Identifica fatores causa (não podem ser influenciados)
fatores_causa = df_fatores.loc[
    df_fatores['Categoria'].isin(['Fator Externo', 'Qualitativo']), 'Fator'
].tolist()

# Cria matriz de relações e dicionário de artigos
artigos = df_fatores.set_index('Fator')['Artigo'].to_dict()
M = df_relacao.set_index('Fatores').loc[:, [c for c in df_relacao.columns if c in df_relacao['Fatores'].values]]

# Gera perguntas de influência (apenas triângulo inferior com 'sim')
perguntas = []
for i, f1 in enumerate(M.index):
    for j, f2 in enumerate(M.columns):
        if j >= i or str(M.iat[i, j]).strip().lower() != 'sim': continue
        a1, a2 = artigos.get(f1, ''), artigos.get(f2, '')

        # f1 -> f2
        if f2 not in fatores_causa and f1 != 'Sucesso do Lançamento':
            perguntas.append({'f1': f1, 'f2': f2, 'pergunta': f"O quanto **{a1} {f1}** influencia **{a2} {f2}**?"})

        # f2 -> f1
        if f1 not in fatores_causa and f2 != 'Sucesso do Lançamento':
            perguntas.append({'f1': f2, 'f2': f1, 'pergunta': f"O quanto **{a2} {f2}** influencia **{a1} {f1}**?"})

df_perguntas = pd.DataFrame(perguntas)

# Ordena perguntas e adiciona IDs
df_perguntas = (df_perguntas
    .merge(df_fatores[['Fator', 'Ordem']], left_on='f1', right_on='Fator')
    .drop(columns=['Fator'])
    .merge(df_fatores[['Fator', 'Ordem']], left_on='f2', right_on='Fator', suffixes=('_f1', '_f2'))
    .drop(columns=['Fator'])
    .sort_values(['Ordem_f1', 'Ordem_f2'])
    .reset_index(drop=True))

df_perguntas['id'] = 'P' + (df_perguntas.index + 1).astype(str).str.zfill(2)

print(f"✅ Carregamento concluído: {len(df_fatores)} fatores, {len(df_perguntas)} perguntas geradas")

✅ Carregamento concluído: 12 fatores, 50 perguntas geradas


In [33]:
# =============================================================================
# 2. BUSCA E PROCESSAMENTO DAS RESPOSTAS DO GOOGLE SHEETS
# =============================================================================

# Conecta ao Google Sheets
creds, _ = google.auth.default(scopes=['https://www.googleapis.com/auth/spreadsheets'])
gc = gspread.authorize(creds)
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1utzXUl0Kr405S1simj41rLYMAmFKOlA16SSejq1ELbQ/edit?resourcekey=&gid=859696412#gid=859696412').worksheet('export')

# Carrega e filtra dados das respostas
df_raw = pd.DataFrame(sheet.get_all_records())
cols_perguntas = [c for c in df_raw.columns if c.startswith('P') and c[1:].isdigit()]
df_respostas = df_raw[cols_perguntas].apply(pd.to_numeric, errors='coerce')

# Transforma para formato longo (expert_id, pergunta, resposta)
df_long = (df_respostas
    .melt(var_name='pergunta', value_name='resposta', ignore_index=False)
    .reset_index()
    .rename(columns={'index': 'expert_id'})
    .merge(df_perguntas[['id', 'f1', 'f2']], left_on='pergunta', right_on='id', how='left')
    .drop(columns=['id'])
    .rename(columns={'f1': 'factor_source', 'f2': 'factor_target', 'resposta': 'influence_score'})
    [['expert_id', 'factor_source', 'factor_target', 'influence_score']])

print(f"✅ Respostas carregadas: {len(df_long)} registros de {df_long['expert_id'].nunique()} especialistas")

✅ Respostas carregadas: 850 registros de 17 especialistas


In [35]:
# =============================================================================
# 3. CRIAÇÃO DA MATRIZ COMPLETA E EXPORTAÇÃO
# =============================================================================

# Cria esqueleto com todas as combinações possíveis de fatores
from itertools import product
esqueleto = pd.DataFrame(list(product(df_fatores['Fator'], df_fatores['Fator'])),
                        columns=['factor_source', 'factor_target'])
esqueleto['influence_score'] = 0

# Para cada especialista, completa respostas faltantes com 0
respostas_completas = []
for expert_id in df_long['expert_id'].unique():
    expert_data = df_long[df_long['expert_id'] == expert_id]
    faltantes = (esqueleto
        .merge(expert_data, on=['factor_source', 'factor_target'], how='left', indicator=True)
        .query('_merge == "left_only"')
        .drop(columns=['influence_score_y', '_merge']))
    faltantes['expert_id'] = expert_id
    faltantes['influence_score'] = 0
    respostas_completas.extend([expert_data, faltantes[['expert_id', 'factor_source', 'factor_target', 'influence_score']]])

df_final = (pd.concat(respostas_completas, ignore_index=True)
    .sort_values(['expert_id', 'factor_source', 'factor_target'])
    .reset_index(drop=True))

# Exporta resultado
output_path = Path('../Delphi/inputs/expert_responses.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)
df_final.to_csv(output_path, index=False)

print(f"💾 Arquivo exportado: {output_path.absolute()}")
print(f"📊 Dimensões: {df_final.shape[0]} linhas × {df_final.shape[1]} colunas")
print(f"👥 Especialistas: {df_final['expert_id'].nunique()}")
print(f"🔢 Scores não-zero: {(df_final['influence_score'] > 0).sum()}")

💾 Arquivo exportado: /Users/joaovieira/Documents/TG/codigos-TG/forms/../Delphi/inputs/expert_responses.csv
📊 Dimensões: 2448 linhas × 4 colunas
👥 Especialistas: 17
🔢 Scores não-zero: 616
